In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Breast Cancer Analysis



## Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import sklearn.metrics as metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

## Read Data

In [ ]:
#I called our dataset and visualized it with the data.head() command.
data=pd.read_csv("/kaggle/input/breast-cancer/data.csv",sep=",")
data.head()


## Editing Data

In [ ]:
#569 rows and 33 columns.
data.shape

In [ ]:
#I decided whether it is true or false by looking at the types in the data set.
data.info()

In [ ]:
#Statistical data brought.
data.describe()

In [ ]:
#Looked for empty data, lost data, missing value.
data.isna().sum()

## Data Preprocessing

In [ ]:
#I looked at the previous line for missing value and found it.
#A column name Unnamed: 32, which contain all null values. So I can drop it.
data.drop(['Unnamed: 32'],axis=1,inplace= True)
#The ID column would not help us contributing to predict about the cancer. I might as well drop it.
data=data.drop('id',axis=1)
data.head()


In [ ]:
#Now there are 31 columns.
data.shape

In [ ]:
# I can check the column has been droped
data.columns
# This gives the column name which is present in our data no Unnamed: 32 and id are not now there. 

In [ ]:
le = LabelEncoder()
data['diagnosis']=le.fit_transform(data['diagnosis'])

In [ ]:
#I now see 0 and 1 instead of "M" and "B"
data

#### We convert our diagnosis property to int values.

#### M --> 1

#### B --> 0

## Data Visualization

In [ ]:
#After checking our data, we can move on to visualization.
#I looked at the numbers of malign and benign patients.
data["diagnosis"].value_counts()


#### We look at how many benign, how many malign tumors there are

#### B (0) --> Benign (357)

#### M (1) --> Malignant (212)

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1, 2, 1)
sns.countplot(x="diagnosis", data=data, palette='Purples')
plt.title("Counts of Diagnosis")


plt.subplot(1, 2, 2)
label=["0","1"]
color=["#b3b3cc","#666699"]
data["diagnosis"].value_counts().plot.pie(explode=[0,0.05],autopct='%1.1f%%',shadow=True,labels=label,colors=color)
plt.show() 





In [ ]:
#Feature Importance

X = data.drop(['diagnosis'], axis = 1)
Y = data['diagnosis']

model = ExtraTreesClassifier(random_state=0)
model.fit(X,Y)
print(model.feature_importances_) 
#plot graph of feature importances for better visualization
color="#666699"
plt.figure(figsize = (8,10))
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(30).plot(kind='bar',color=color)
plt.title('Feature importances for \nBreast Cancer ')
plt.show()

#### I saw concave points worst affect more diagnosis.

## Correlation Analysis

In [ ]:
#The measure of the relationship between them.
data.corr()

In [ ]:
# Heatmap
plt.figure(figsize=(20,20))
sns.heatmap(data.corr(),annot = True, cmap = 'Purples');
plt.title("Correlation Between Features")

In [ ]:
#I can say that as the area_mean and radius_mean values increase their is a higher chance being diagnosed with Cancer.
#I can say that as the area_mean and perimeter_mean values increase their is a higher chance being diagnosed with Cancer.
plt.figure(figsize = (10,5))
plt.subplot(1, 2, 1)
sns.scatterplot(x = 'area_mean', y = 'radius_mean', hue = 'diagnosis', data = data)
plt.show()
plt.figure(figsize = (10,5))
plt.subplot(1, 2, 2)
sns.scatterplot(x = 'area_mean', y = 'perimeter_mean', hue = 'diagnosis', data = data)
plt.show()


## Train-Test Split

In [ ]:
#Now split our data into two as train and test.
X = data.drop(['diagnosis'], axis = 1)
Y = data['diagnosis']


In [ ]:
# Splitting the data into training and test sets.
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 0)


In [ ]:
#applying feature scaling may improve accuracy.
#Standardizing training data.
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
X_train

## Classification Task: Logistic Regression

In [ ]:
#log reg.
log_reg = LogisticRegression(random_state=1, max_iter=100).fit(X_train, Y_train)
predict_log=log_reg.predict(X_test)
predict_log

In [ ]:
acc_logreg = accuracy_score(Y_test, predict_log)
print("Logistic Regression Classifier Accuracy:",metrics.accuracy_score(Y_test, predict_log))

In [ ]:
ConfMatrix = confusion_matrix(Y_test,log_reg.predict(X_test))
sns.heatmap(ConfMatrix,annot=True, cmap="Purples", fmt="d", 
            xticklabels = ['0', '1'], yticklabels = ['0', '1'])

plt.title("Confusion Matrix");

In [ ]:
print("Confusion Matrix: \n", confusion_matrix(Y_test, predict_log))
print('\n')
print(classification_report(Y_test, predict_log))

## Classification Task: Random Forest

In [ ]:
#randomforest
forest=RandomForestClassifier(n_estimators=20,criterion="entropy",random_state=0)
forest.fit(X_train,Y_train)

In [ ]:
#predict for forest classifier.
forest_pred=forest.predict(X_test)
forest_pred

In [ ]:

acc_forest = accuracy_score(Y_test, forest_pred)
print("Random Forest Classifier Accuracy:",metrics.accuracy_score(Y_test, forest_pred))

In [ ]:
ConfMatrix = confusion_matrix(Y_test,forest.predict(X_test))
sns.heatmap(ConfMatrix,annot=True, cmap="Purples", fmt="d", 
            xticklabels = ['0', '1'], yticklabels = ['0', '1'])

plt.title("Confusion Matrix");

In [ ]:
print("Confusion Matrix: \n", confusion_matrix(Y_test, forest_pred))
print('\n')
print(classification_report(Y_test, forest_pred))

## Classification Task: KNN

In [ ]:
#knn
KNN=KNeighborsClassifier(n_neighbors=3)
KNN.fit(X_train,Y_train)

In [ ]:
#predict for KNN.
knn_pred=KNN.predict(X_test)

In [ ]:
acc_knn = accuracy_score(Y_test, knn_pred)

print("KNN Classifier Accuracy:",metrics.accuracy_score(Y_test, knn_pred))

In [ ]:
ConfMatrix = confusion_matrix(Y_test,KNN.predict(X_test))
sns.heatmap(ConfMatrix,annot=True, cmap="Purples", fmt="d", 
            xticklabels = ['0', '1'], yticklabels = ['0', '1'])

plt.title("Confusion Matrix");

In [ ]:
print("Confusion Matrix: \n", confusion_matrix(Y_test, knn_pred))
print('\n')
print(classification_report(Y_test, knn_pred))

## Classification Task: Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train,Y_train)




In [ ]:
nb_pred=nb.predict(X_test)
nb_pred

In [ ]:
acc_nb = accuracy_score(Y_test, nb_pred)
print("NB Classifier Accuracy:",metrics.accuracy_score(Y_test, nb_pred))

In [ ]:
ConfMatrix = confusion_matrix(Y_test,nb.predict(X_test))
sns.heatmap(ConfMatrix,annot=True, cmap="Purples", fmt="d", 
            xticklabels = ['0', '1'], yticklabels = ['0', '1'])

plt.title("Confusion Matrix");

In [ ]:
print("Confusion Matrix: \n", confusion_matrix(Y_test, nb_pred))
print('\n')
print(classification_report(Y_test, nb_pred))

## Classification Task: XGBoost

In [ ]:
xgb=XGBClassifier(random_state=0,booster="gbtree")
xgb.fit(X_train,Y_train)

In [ ]:
xgb_pred=xgb.predict(X_test)
xgb_pred

In [ ]:
acc_xgb = accuracy_score(Y_test, xgb_pred)
print("XGB Classifier Accuracy:",metrics.accuracy_score(Y_test, xgb_pred))

In [ ]:
ConfMatrix = confusion_matrix(Y_test,xgb.predict(X_test))
sns.heatmap(ConfMatrix,annot=True, cmap="Purples", fmt="d", 
            xticklabels = ['0', '1'], yticklabels = ['0', '1'])

plt.title("Confusion Matrix");

In [ ]:
print("Confusion Matrix: \n", confusion_matrix(Y_test, xgb_pred))
print('\n')
print(classification_report(Y_test, xgb_pred))

## Compare Performance

In [ ]:
model = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'Random Forest Classifier', 
             'Naive Bayes', 'XgBoost'],
    'Accuracy': [acc_logreg, acc_knn, acc_forest,acc_nb,acc_xgb]
})

model.sort_values(by = 'Accuracy', ascending = False)

In [ ]:
 sns.barplot(x = model['Accuracy'], y = model['Model'], palette='Purples');

#### The most efficient analysis method is logistic regression.
#### I hope you like it. Please upvote the notebook. Thanks :))